In [ ]:
import random

In [22]:
class ChatBot:
    
    def __init__(self):
        
        self.tasks = {0: ['weather', 'rain', 'sunny', 'cloudy', 'snow'],
                      1: ['restaurant', 'eat', 'food', 'hungry'],
                      2: ['tram', 'bus', 'train', 'boat']}


    def handle_user_request(self, user_input):
        
        user_input = user_input.lower()
        
        for task, keywords in self.tasks.items():
            
            if any(keyword in user_input for keyword in keywords):
                return self.get_task(task, user_input)
            
        return 'sorry, I no speak english' #if no keywords are found
    

    def get_task(self, task, user_input):

        if task == 0:
            return self.weather_forecast(user_input)
        
        elif task == 1:
            return self.suggest_restaurant(user_input)
        
        elif task == 2: 
            return self.find_next_transport(user_input)
        
        return 'you are using the default version. please upgrade to get all premium benefits'


    def weather_forecast(self, user_input):

        keyword = next((keyword for keyword in self.tasks[0] if keyword in user_input), None)
        
        if 'weather' not in user_input:

            chance = random.choice(['high', 'low'])

            return f'The chance of {keyword} today is {chance}.'

        else: 

            forecasts = ['sunny', 'rainy', 'cloudy', 'snowy']

            return f'The weather is going to be {random.choice(forecasts)} today.'
    

    def suggest_restaurant(self, user_input):

        restaurants = ['Kårrestaurangen', 'Bulten', 'Linsen', 'Wijkanders']

        return f'I recommend you should eat at {random.choice(restaurants)}.'
    

    def find_next_transport(self, user_input):

        times = [5, 10, 15, 20, 25]


        return f'The next tram arrives in {random.choice(times)} minutes.'


In [24]:
bot = ChatBot()

request = 'Is it going to be sunny today?'
request2 = 'How is the weather today?'

print('Request:', request)
print('\nAnswer:', bot.handle_user_request(request))

Request: Is it going to be sunny today?

Answer: The chance of sunny today is low.


### More advanved...

In [25]:
#import nltk
#nltk.download('punkt')
#nltk.download('stopwords')

import numpy as np
import pandas as pd
import tensorflow as tf
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences


In [7]:
training_df = pd.read_csv('WikiQA-train.tsv', sep='\t')

In [8]:
def preprocess_text(text):

    text = text.lower()

    text = ''.join([char for char in text if char not in string.punctuation])

    words = word_tokenize(text)

    stop_words = set(stopwords.words('english'))
    
    filtered_words = [word for word in words if word not in stop_words]
    
    return ' '.join(filtered_words)



In [67]:
filtered_training = training_df.copy()[training_df['Label'] == 1]

#filtered_training['Question'] = filtered_training['Question'].apply(preprocess_text)
#filtered_training['Sentence'] = filtered_training['Sentence'].apply(preprocess_text)

In [100]:
class SimpleAssistant:
    def __init__(self, data):
        self.data = data

    def find_answer(self, question):

        question_keywords = preprocess_text(question).split()


        def contains_keywords(answer):
            matches = sum(keyword in answer['Question'] for keyword in question_keywords)
            return matches >= 3

        matches = self.data[self.data.apply(contains_keywords, axis=1)]
        
        for _, row in matches.iterrows():
            if row['Label'] == 1:
                return row['Sentence']
            
        return 'I am not able to answer that question.'



In [101]:
assistant = SimpleAssistant(filtered_training)

question = 'When was Thomas Jefferson president?'

print(assistant.find_answer(question))

Thomas Jefferson (April 13, 1743 (April 2, 1743 O.S. ) – July 4, 1826) was an American Founding Father , the principal author of the Declaration of Independence (1776) and the third President of the United States (1801–1809).


In [103]:
question = 'Which languages are spoken in South Africa?'

print(assistant.find_answer(question))

South Africa has eleven official languages : Afrikaans , English , Ndebele , Northern Sotho , Sotho , Swazi , Tswana , Tsonga , Venda , Xhosa and Zulu .
